# Decision Making: Analytic Hierarchy Process by T.L Satty

Study Notes

# Problem:

In [3]:
import pandas as pd
import numpy as np

# Define the criteria based on the image content
criteria = ["Cost Evolution", "Environmental Impact", "Transporting and Packaging", "Infrastructure Alignment"]

rows = []
rows.append({"Criteria": "Cost Evolution", "Australia": "4,84 euros per kg", "Chilli": "4,86 per kg", "Norway": "2-5 euros per kg", "UAE": "5,36 euro per kg"})
rows.append({"Criteria": "Environmental Impact", "Australia": "High", "Chilli": "High", "Norway": "Low", "UAE": "Medium"})
rows.append({"Criteria": "Transporting and Packaging", "Australia": "Critical geography Low possibility", "Chilli": "Critical geography Low possibility", "Norway": "Good Location High possibility", "UAE": "Good location Low possibility"})
rows.append({"Criteria": "Infrastructure Alignment", "Australia": "Low possibility", "Chilli": "low possibility", "Norway": "High possibility", "UAE": "Low possibility"})


problem_df = pd.DataFrame(rows)

problem_df

,Criteria,Australia,Chilli,Norway,UAE
0,Cost Evolution,"4,84 euros per kg","4,86 per kg",2-5 euros per kg,"5,36 euro per kg"
1,Environmental Impact,High,High,Low,Medium
2,Transporting and Packaging,Critical geography Low possibility,Critical geography Low possibility,Good Location High possibility,Good location Low possibility
3,Infrastructure Alignment,Low possibility,low possibility,High possibility,Low possibility


# Step 1: Making pairwise comparison of each criteria / sub-criteria

In [18]:
# Create a pandas DataFrame from the pairwise comparison matrix provided
criteria = ["Cost Evolution", "Environmental Impact", "Transporting and Packaging", "Infrastructure Alignment"]
pairwise_values = [
    [1, 1/3, 1/6, 1/4],
    [3, 1  , 1/2, 1/3],
    [6, 2  , 1  , 1/2],
    [4, 3  , 2  , 1  ]
]

# Construct the df
pairwise_df = pd.DataFrame(pairwise_values, columns=criteria, index=criteria)

# Construct the df with a total row
pairwise_total_df = pairwise_df.copy()
pairwise_total_df.loc['Total'] = pairwise_df.sum(axis=0)

pairwise_df

,Cost Evolution,Environmental Impact,Transporting and Packaging,Infrastructure Alignment
Cost Evolution,1,0.333333,0.166667,0.250000
Environmental Impact,3,1.000000,0.500000,0.333333
Transporting and Packaging,6,2.000000,1.000000,0.500000
Infrastructure Alignment,4,3.000000,2.000000,1.000000


In [19]:
pairwise_total_df

,Cost Evolution,Environmental Impact,Transporting and Packaging,Infrastructure Alignment
Cost Evolution,1.0,0.333333,0.166667,0.250000
Environmental Impact,3.0,1.000000,0.500000,0.333333
Transporting and Packaging,6.0,2.000000,1.000000,0.500000
Infrastructure Alignment,4.0,3.000000,2.000000,1.000000
Total,14.0,6.333333,3.666667,2.083333


In [20]:
# Normalizing the matrix:
column_sums = pairwise_df.sum()
normalized_df = pairwise_df / column_sums

normalized_df

,Cost Evolution,Environmental Impact,Transporting and Packaging,Infrastructure Alignment
Cost Evolution,0.071429,0.052632,0.045455,0.12
Environmental Impact,0.214286,0.157895,0.136364,0.16
Transporting and Packaging,0.428571,0.315789,0.272727,0.24
Infrastructure Alignment,0.285714,0.473684,0.545455,0.48


In [21]:
# Obtaining the relative priority - general of each criterion:
relative_priority = normalized_df.mean(axis=1)

relative_priority_general_df = pd.DataFrame(relative_priority, columns=['Relative Priority - General'])
relative_priority_general_df

,Relative Priority - General
Cost Evolution,0.072379
Environmental Impact,0.167136
Transporting and Packaging,0.314272
Infrastructure Alignment,0.446213


In [22]:
print('The total sum of the normalized matrix is:')
print(normalized_df.sum(axis=0))
print('-'*10)
print('The total sum relative priority column is:')
print(relative_priority_general_df.sum(axis=0))

The total sum of the normalized matrix is:
Cost Evolution                1.0
Environmental Impact          1.0
Transporting and Packaging    1.0
Infrastructure Alignment      1.0
dtype: float64
----------
The total sum relative priority column is:
Relative Priority - General    1.0
dtype: float64


## Obtaining the vector of weights:

In [23]:
# Calculate the vector of weights by performing a dot product operation
weights_vector = pairwise_df.dot(relative_priority)

weights_vector_df = pd.DataFrame(weights_vector, columns=['Vector of Weights'])
weights_vector_df

,Vector of Weights
Cost Evolution,0.292023
Environmental Impact,0.690146
Transporting and Packaging,1.305923
Infrastructure Alignment,1.865680


## Obtaining the consistency vector:

In [24]:
# Calculate the consistency vector by dividing each weight by its relative priority
consistency_vector = weights_vector / relative_priority

consistency_vector_df = pd.DataFrame(consistency_vector, columns=['Consistency Vector'])
consistency_vector_df

,Consistency Vector
Cost Evolution,4.034651
Environmental Impact,4.129246
Transporting and Packaging,4.155390
Infrastructure Alignment,4.181140


## Obtaining the value λmax:

In [25]:
# Calculate λmax by taking the mean of the consistency vector
lambda_max = consistency_vector.mean()

lambda_max

4.125106670462181

## Obtaining the consistency index CI:

The Consistency Index (CI) is calculated using the formula:

$$
CI = \frac{\lambda_{\text{max}} - n}{n - 1}
$$

where:

- $\lambda_{\text{max}}$ is the largest eigenvalue of the pairwise comparison matrix,
- \( n \) is the number of criteria.




In [26]:
CI = (lambda_max - len(criteria)) / (len(criteria) - 1)

CI

0.04170222348739353

## Obtaining the Consistency Ratio CR:

The Consistency Ratio (CR) is obtained using the formula:

$$
CR = \frac{CI}{ACI}
$$

where:

- \( CI \) is the Consistency Index,
- \( ACI \) is the Average Consistency Index for a large number of pairwise comparisons.


In [27]:
# ACI Table found int prof. Rasche's slides
aci_table = pd.DataFrame({
    'N': [3, 4, 5, 6, 7, 8],
    'ACI': [0.58, 0.90, 1.12, 1.24, 1.32, 1.41]
})

aci_table # we select the ACI value for N = 4 which is 0.90

,N,ACI
0,3,0.58
1,4,0.90
2,5,1.12
3,6,1.24
4,7,1.32
5,8,1.41


In [28]:
CR = CI / 0.90
CR

0.046335803874881704

# Step 2: Comparison criterion according to the alternatives

## Cost Evolution Matrix

In [29]:
# Extracted data from the cost evolution image
cost_evolution_data = {
    'Australia': [1, 1, 4, 1/3],
    'Chile': [1, 1, 4, 1/2],
    'Norway': [1/4, 1/4, 1, 1/4],
    'UAE': [3, 2, 4, 1]
}

# Create a pandas DataFrame from the cost evolution data
countries = ["Australia", "Chile", "Norway", "UAE"]
cost_evolution_df = pd.DataFrame(cost_evolution_data, index=countries)

# Display the DataFrame
cost_evolution_df


,Australia,Chile,Norway,UAE
Australia,1.000000,1.0,0.25,3
Chile,1.000000,1.0,0.25,2
Norway,4.000000,4.0,1.00,4
UAE,0.333333,0.5,0.25,1


In [30]:
#normalizing the matrix
# Calculate the sum of each column
column_sums = cost_evolution_df.sum()

# Normalize the DataFrame by dividing each element by the sum of its column
cost_evolution_normalized_df = cost_evolution_df / column_sums

# Display the normalized DataFrame
cost_evolution_normalized_df

,Australia,Chile,Norway,UAE
Australia,0.157895,0.153846,0.142857,0.3
Chile,0.157895,0.153846,0.142857,0.2
Norway,0.631579,0.615385,0.571429,0.4
UAE,0.052632,0.076923,0.142857,0.1


In [31]:
# Obtaining the relative priority of each criterion:
# Calculate the mean of each row to get the relative priority of each criterion
cost_evolution_relative_priority = cost_evolution_normalized_df.mean(axis=1)

cost_evolution_relative_priority_df = pd.DataFrame(cost_evolution_relative_priority, columns=['Relative Priority - Cost Evolution'])

cost_evolution_relative_priority_df

,Relative Priority - Cost Evolution
Australia,0.188650
Chile,0.163650
Norway,0.554598
UAE,0.093103


## Environmental impact Values

In [32]:
import pandas as pd

# Recreate the data for environmental impact based on the provided pairwise comparison image
environmental_impact_data = {
    'Australia': [1, 1, 1/3, 1/2],
    'Chile':     [1, 1, 1/3, 1/2],
    'Norway':    [3, 3, 1  , 2  ],
    'UAE':       [2, 2, 1/2, 1  ]
}

# Create a pandas DataFrame from the environmental impact data
countries = ["Australia", "Chile", "Norway", "UAE"]
environmental_impact_df = pd.DataFrame(environmental_impact_data, index=countries)

# Display the DataFrame
environmental_impact_df = environmental_impact_df.T
environmental_impact_df


,Australia,Chile,Norway,UAE
Australia,1.0,1.0,0.333333,0.5
Chile,1.0,1.0,0.333333,0.5
Norway,3.0,3.0,1.000000,2.0
UAE,2.0,2.0,0.500000,1.0


In [33]:
#normalizing the matrix
# Calculate the sum of each column
column_sums = environmental_impact_df.sum()

# Normalize the DataFrame by dividing each element by the sum of its column
environmental_impact_normalized_df = environmental_impact_df / column_sums

# Display the normalized DataFrame
environmental_impact_normalized_df

,Australia,Chile,Norway,UAE
Australia,0.142857,0.142857,0.153846,0.125
Chile,0.142857,0.142857,0.153846,0.125
Norway,0.428571,0.428571,0.461538,0.500
UAE,0.285714,0.285714,0.230769,0.250


In [34]:
# Obtaining the relative priority of each criterion:
# Calculate the mean of each row to get the relative priority of each criterion
environmental_impact_relative_priority = environmental_impact_normalized_df.mean(axis=1)

environmental_impact_relative_priority_df = pd.DataFrame(environmental_impact_relative_priority, columns=['Relative Priority - Environmental Impact'])

environmental_impact_relative_priority_df

,Relative Priority - Environmental Impact
Australia,0.141140
Chile,0.141140
Norway,0.454670
UAE,0.263049


## Transporting and Packaging scored matrix

In [35]:
# Recreate the data for Transporting and Packaging based on the provided pairwise comparison image
transporting_packaging_data = {
    'Australia':[1, 1, 1/4, 1/4],
    'Chile':    [1, 1, 1/4, 1/4],
    'Norway':   [4, 4, 1  , 2],
    'UAE':      [4, 4, 1/2, 1] #[4, 4, 1/2, 1]
}

# Create a pandas DataFrame from the transporting and packaging data
countries = ["Australia", "Chile", "Norway", "UAE"]
transporting_packaging_df = pd.DataFrame(transporting_packaging_data, index=countries)

# Display the DataFrame
transporting_packaging_df = transporting_packaging_df.T
transporting_packaging_df


,Australia,Chile,Norway,UAE
Australia,1.0,1.0,0.25,0.25
Chile,1.0,1.0,0.25,0.25
Norway,4.0,4.0,1.00,2.00
UAE,4.0,4.0,0.50,1.00


In [36]:
#normalizing the matrix
# Calculate the sum of each column
column_sums = transporting_packaging_df.sum()

# Normalize the DataFrame by dividing each element by the sum of its column
transporting_packaging_normalized_df = transporting_packaging_df / column_sums

# Display the normalized DataFrame
transporting_packaging_normalized_df

,Australia,Chile,Norway,UAE
Australia,0.1,0.1,0.125,0.071429
Chile,0.1,0.1,0.125,0.071429
Norway,0.4,0.4,0.500,0.571429
UAE,0.4,0.4,0.250,0.285714


In [37]:
#   Obtaining the relative priority of each criterion:
# Calculate the mean of each row to get the relative priority of each criterion
transporting_packaging_relative_priority = transporting_packaging_normalized_df.mean(axis=1)

transporting_packaging_relative_priority_df = pd.DataFrame(transporting_packaging_relative_priority, columns=['Relative Priority - Transporting and Packaging'])

transporting_packaging_relative_priority_df

,Relative Priority - Transporting and Packaging
Australia,0.099107
Chile,0.099107
Norway,0.467857
UAE,0.333929


## Infrastructure Alignment



In [38]:
# Recreate the data for Infrastructure Alignment based on the provided pairwise comparison image
infrastructure_alignment_data = {
    'Australia':[1, 1, 1/3, 1],
    'Chile':    [1, 1, 1/3, 1],
    'Norway':   [3, 3, 1  , 3],
    'UAE':      [1, 1, 1/3, 1]
}

# Create a pandas DataFrame from the infrastructure alignment data
countries = ["Australia", "Chile", "Norway", "UAE"]
infrastructure_alignment_df = pd.DataFrame(infrastructure_alignment_data, index=countries)

# Display the DataFrame
infrastructure_alignment_df.T


,Australia,Chile,Norway,UAE
Australia,1.0,1.0,0.333333,1.0
Chile,1.0,1.0,0.333333,1.0
Norway,3.0,3.0,1.000000,3.0
UAE,1.0,1.0,0.333333,1.0


In [39]:
#normalizing the matrix
# Calculate the sum of each column
column_sums = infrastructure_alignment_df.sum()

# Normalize the DataFrame by dividing each element by the sum of its column
infrastructure_alignment_normalized_df = infrastructure_alignment_df / column_sums

# Display the normalized DataFrame  
infrastructure_alignment_normalized_df

,Australia,Chile,Norway,UAE
Australia,0.3,0.3,0.3,0.3
Chile,0.3,0.3,0.3,0.3
Norway,0.1,0.1,0.1,0.1
UAE,0.3,0.3,0.3,0.3


In [40]:
#   Obtaining the relative priority of each criterion:
# Calculate the mean of each row to get the relative priority of each criterion
infrastructure_alignment_relative_priority = infrastructure_alignment_normalized_df.mean(axis=1)

infrastructure_alignment_relative_priority_df = pd.DataFrame(infrastructure_alignment_relative_priority, columns=['Relative Priority - Infrastructure Alignment'])

infrastructure_alignment_relative_priority_df

,Relative Priority - Infrastructure Alignment
Australia,0.3
Chile,0.3
Norway,0.1
UAE,0.3


# Obtaining the compound priority for alternatives: Australia, Chile, Norway & UAE

In [41]:
# Obtaining the compound priority for alternatives: Australia, Chile, Norway & UAE

# merge all the relative priority dataframes
relative_priority_df = pd.concat([cost_evolution_relative_priority_df, environmental_impact_relative_priority_df, transporting_packaging_relative_priority_df, infrastructure_alignment_relative_priority_df], axis=1)

relative_priority_df

,Relative Priority - Cost Evolution,Relative Priority - Environmental Impact,Relative Priority - Transporting and Packaging,Relative Priority - Infrastructure Alignment
Australia,0.188650,0.141140,0.099107,0.3
Chile,0.163650,0.141140,0.099107,0.3
Norway,0.554598,0.454670,0.467857,0.1
UAE,0.093103,0.263049,0.333929,0.3


In [42]:
relative_priority_general_df

,Relative Priority - General
Cost Evolution,0.072379
Environmental Impact,0.167136
Transporting and Packaging,0.314272
Infrastructure Alignment,0.446213


In [43]:
import numpy as np

# Convert the DataFrames to numpy arrays
array1 = relative_priority_df.to_numpy()
array2 = relative_priority_general_df.to_numpy()

# Reshape array2 to match the shape of array1 for element-wise multiplication
array2_reshaped = np.repeat(array2, repeats=array1.shape[1], axis=1)

# Perform element-wise multiplication and then sum along the axis 1 (columns)
result = np.sum(np.multiply(array1, array2_reshaped), axis=1)

result

array([0.05275658, 0.1176465 , 0.49564646, 0.44178726])

In [46]:
#turn the result into a dataframe with the countries as index
result_df = pd.DataFrame(result, columns=['Relative Priority - Composed'])
result_df.index = countries
result_df

,Relative Priority - Composed
Australia,0.052757
Chile,0.117647
Norway,0.495646
UAE,0.441787


Norway being the highest values is the most suitable choice

# References:

1. [Using the analytic hierarchy process for decision making in engineering applications: Some challenges, February 1995The International Journal of Industrial Engineering: Theory, Applications and Practice 2(1):35-44](https://www.researchgate.net/publication/241416054_Using_the_analytic_hierarchy_process_for_decision_making_in_engineering_applications_Some_challenges)

2. Lecture Notes: Decision Making Sem. 3 (Decision Making Methods – Malte Weber / Rebecca Rasche)